In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
#Read the ods File
path = "../Raw Data/apr19.ods"
df = pd.read_excel(path, engine="odf")
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,11859,79.06%,4049,48018312,47856327,109087678
1,TheGrandHotel,Taipei,0,0,NaN,0,0,0,0
2,國賓⼤飯店,Taipei,422,8839,69.82%,3947,34889303,65307831,106694543
3,TheAmbassadorHotel-Taipei,Taipei,0,0,NaN,0,0,0,0
4,台北華國⼤飯店,Taipei,326,8023,82.03%,3081,24720379,7719544,34209508


In [3]:
#Clean Data 
df = pd.DataFrame(df, columns=['Hotel','Area','Room','Demand','Occupancy','ADR','Room_Revenue','F&B_Revenue','Total_Revenue'])
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,11859,79.06%,4049,48018312,47856327,109087678
1,TheGrandHotel,Taipei,0,0,NaN,0,0,0,0
2,國賓⼤飯店,Taipei,422,8839,69.82%,3947,34889303,65307831,106694543
3,TheAmbassadorHotel-Taipei,Taipei,0,0,NaN,0,0,0,0
4,台北華國⼤飯店,Taipei,326,8023,82.03%,3081,24720379,7719544,34209508


In [4]:
df = df.dropna()
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,11859,79.06%,4049,48018312,47856327,109087678
2,國賓⼤飯店,Taipei,422,8839,69.82%,3947,34889303,65307831,106694543
4,台北華國⼤飯店,Taipei,326,8023,82.03%,3081,24720379,7719544,34209508
6,華泰王⼦⼤飯店,Taipei,213,4455,69.72%,2796,12457926,7290588,22579152
8,國王⼤飯店,Taipei,97,1823,62.65%,1624,2960928,736263,3787611
...,...,...,...,...,...,...,...,...,...
246,台東桂⽥喜來登酒店,Others,290,3510,40.34%,2939,10316994,11408339,22560171
248,寶華⼤飯店,Others,78,1032,44.10%,1636,1688864,0,1690864
250,⻑榮桂冠酒店(基隆),Others,140,3432,81.71%,2424,8317685,5034616,16157022
252,鈺通⼤飯店,Others,120,1840,51.11%,1810,3331263,3874212,7205475


In [5]:
df.loc[df['Hotel']=='華泰王⼦⼤飯店']

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
6,華泰王⼦⼤飯店,Taipei,213,4455,69.72%,2796,12457926,7290588,22579152


In [6]:
print(df.dtypes)

Hotel            object
Area             object
Room              int64
Demand            int64
Occupancy        object
ADR               int64
Room_Revenue      int64
F&B_Revenue       int64
Total_Revenue     int64
dtype: object


df['Demand'] = df['Demand'].replace(',', '', regex=True)
df['ADR'] = df['ADR'].replace(',', '', regex=True)
df['Room_Revenue'] = df['Room_Revenue'].replace(',', '', regex=True)
df['F&B_Revenue'] = df['F&B_Revenue'].replace(',', '', regex=True)
df['Total_Revenue'] = df['Total_Revenue'].replace(',', '', regex=True)

In [7]:
df['Occupancy'] = df['Occupancy'].str.rstrip('%').astype('float') / 100.0

In [8]:
df[df['Occupancy'].isnull()]

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue


In [9]:
df.astype({'Room': 'int64',
          'Demand': 'int64',
          'Occupancy': 'int64',
          'ADR': 'float64',
          'Room_Revenue': 'int64',
          'F&B_Revenue': 'int64',
          'Total_Revenue': 'int64',
          }).dtypes

Hotel             object
Area              object
Room               int64
Demand             int64
Occupancy          int64
ADR              float64
Room_Revenue       int64
F&B_Revenue        int64
Total_Revenue      int64
dtype: object

In [10]:
Day = 30

In [11]:
#Calculate/Add RevPAR and Demand
df['RevPAR'] = df['Occupancy']*df['ADR']
df['Supply'] = df['Room']*Day
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
0,圓⼭⼤飯店,Taipei,500,11859,0.7906,4049,48018312,47856327,109087678,3201.1394,15000
2,國賓⼤飯店,Taipei,422,8839,0.6982,3947,34889303,65307831,106694543,2755.7954,12660
4,台北華國⼤飯店,Taipei,326,8023,0.8203,3081,24720379,7719544,34209508,2527.3443,9780
6,華泰王⼦⼤飯店,Taipei,213,4455,0.6972,2796,12457926,7290588,22579152,1949.3712,6390
8,國王⼤飯店,Taipei,97,1823,0.6265,1624,2960928,736263,3787611,1017.4360,2910
...,...,...,...,...,...,...,...,...,...,...,...
246,台東桂⽥喜來登酒店,Others,290,3510,0.4034,2939,10316994,11408339,22560171,1185.5926,8700
248,寶華⼤飯店,Others,78,1032,0.4410,1636,1688864,0,1690864,721.4760,2340
250,⻑榮桂冠酒店(基隆),Others,140,3432,0.8171,2424,8317685,5034616,16157022,1980.6504,4200
252,鈺通⼤飯店,Others,120,1840,0.5111,1810,3331263,3874212,7205475,925.0910,3600


In [12]:
#Calculate Monthly Room/Supply/Demand/Room Revenue/F&B Revenue/Total Revenue
date = "2019-04-30"
room = round(df["Room"].sum(),0)
supply = round(df["Supply"].sum(),0)
demand = round(df['Demand'].sum(),0)
room_revenue = df["Room_Revenue"].sum()
fb_revenue = df["F&B_Revenue"].sum()
total_revenue = df["Total_Revenue"].sum()

In [13]:
#Calculate Monthly Occupancy/ADR/RevPAR
occ = round((demand/supply),2)
adr = round((room_revenue/demand), 2)
#revpar = round((occ*adr), 2)
revpar = round((room_revenue/supply), 2)

In [14]:
#Append "Total" Row
df = df.append([{'Hotel': 'Total',
                'Area': 'Total',
                'Room': room,
                'Supply': supply,
                'Occupancy': occ,
                'ADR': adr,
                'Room_Revenue': room_revenue,
                'F&B_Revenue': fb_revenue,
                'Total_Revenue': total_revenue,
                'RevPAR': revpar,
                'Demand': demand}], ignore_index=True)
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
124,寶華⼤飯店,Others,78,1032,0.4410,1636.00,1688864,0,1690864,721.4760,2340
125,⻑榮桂冠酒店(基隆),Others,140,3432,0.8171,2424.00,8317685,5034616,16157022,1980.6504,4200
126,鈺通⼤飯店,Others,120,1840,0.5111,1810.00,3331263,3874212,7205475,925.0910,3600
127,陸島酒店,Others,47,405,0.2872,1834.00,742885,61280,838165,526.7248,1410
128,Total,Total,29498,603321,0.6800,3817.93,2303436301,1805482426,4622245655,2602.9300,884940


In [15]:
#Add "Date" Column 
df['Date'] = date
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply,Date
124,寶華⼤飯店,Others,78,1032,0.4410,1636.00,1688864,0,1690864,721.4760,2340,2019-04-30
125,⻑榮桂冠酒店(基隆),Others,140,3432,0.8171,2424.00,8317685,5034616,16157022,1980.6504,4200,2019-04-30
126,鈺通⼤飯店,Others,120,1840,0.5111,1810.00,3331263,3874212,7205475,925.0910,3600,2019-04-30
127,陸島酒店,Others,47,405,0.2872,1834.00,742885,61280,838165,526.7248,1410,2019-04-30
128,Total,Total,29498,603321,0.6800,3817.93,2303436301,1805482426,4622245655,2602.9300,884940,2019-04-30


In [16]:
df['Date'] = pd.to_datetime(df['Date']).dt.to_period('D')

In [17]:
#Format 'Summary'
df['Date'] = df['Date'].dt.to_timestamp()
#df = df.style.format({'occ':'{:.0%}',
                    #'demand':'{:.0f}',
                    #'adr':'{:.2f}',
                    #'revpar':'{:.2f}'})

In [18]:
print(df.dtypes)

Hotel                    object
Area                     object
Room                      int64
Demand                    int64
Occupancy               float64
ADR                     float64
Room_Revenue              int64
F&B_Revenue               int64
Total_Revenue             int64
RevPAR                  float64
Supply                    int64
Date             datetime64[ns]
dtype: object


#Create DataFrame 'Summary'
summary = pd.DataFrame({
    'date':date,
    'room':room,
    'supply':supply,
    'demand':demand,
    'room_revenue': room_revenue, 
    'fb_revenue': fb_revenue,
    'total_revenue': total_revenue,
    'occ': occ,
    'adr': adr,
    'revpar':revpar   
}, index=[0])
summary

#Format Summary
summary['date'] = pd.to_datetime(summary['date']).dt.to_period('M')
sep20 = summary.style.format({'occ':'{:.0%}',
                    'demand':'{:.0f}',
                    'adr':'{:.2f}',
                    'revpar':'{:.2f}'})

sep20

df.to_excel(r'static\data\2020\sep20.xlsx', index = False, encoding='utf-8-sig')

In [19]:
df.to_csv(r'..\static\data\2019\apr19.csv', index = False, encoding='utf-8-sig')

In [20]:
from sqlalchemy import create_engine

In [21]:
connection_string = "postgres:postgres@localhost:5432/hotel_db"
engine = create_engine(f'postgresql://{connection_string}', encoding = 'utf-8')

In [22]:
engine.table_names()

['jan20',
 'oct20',
 'nov20',
 'dec20',
 'combine20',
 'dec19',
 'nov19',
 'oct19',
 'sep19',
 'jun19',
 'aug19',
 'jul19',
 'may19',
 'apr19',
 'mar19',
 'feb19',
 'jan19',
 'feb20',
 'mar20',
 'apr20',
 'may20',
 'jun20',
 'jul20',
 'aug20',
 'sep20']

In [23]:
df.to_sql(name='apr19', con=engine, if_exists='append', index=False)